In [1]:
import pandas as pd
import numpy as np
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import category_encoders as ce
import pickle
from sklearn.svm import SVC
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
train = pd.read_csv("../data/sample_train.txt", delimiter="\t")
test = pd.read_csv("../data/test_id.txt", delimiter="\t")

In [3]:
df = pd.concat([train, test], axis=0)

In [4]:
risk = pd.read_csv("../features/risk/risk.csv")
df = df.merge(risk, on="id", how="left")

In [5]:
symbol = pd.read_csv("../features/symbol/symbol.csv")
df = df.merge(symbol, on="id", how="left")

In [6]:
# dp_emb = pd.read_csv("../features/graph/graph_filtered_64.emb")
# df = df.merge(dp_emb, on="id", how="left")

In [7]:
dp_emb = pd.read_csv("../features/graph/deepwalk_256_filtered.emb")
df = df.merge(dp_emb, on="id", how="left")

In [8]:
dp_emb.shape

(28442, 257)

In [9]:
# nv_emb = pd.read_csv("../data/edge/node2vec_64.emb")
# df = df.merge(nv_emb, on="id", how="left")

In [10]:
graph_info = pd.read_csv("../features/graph/graph_info.csv")
df = df.merge(graph_info, on="id", how="left")

In [11]:
# app_pca = pd.read_csv("../data/app/app_pca_16.csv")
# df = df.merge(app_pca, on="id", how="left")

app_lda = pd.read_csv("../features/app/app_lda_16.csv")
df = df.merge(app_lda, on="id", how="left")

# app_lda = pd.read_csv("../data1/app/app_lda_16.csv")
# df = df.merge(app_lda, on="id", how="left")

# app_nmf = pd.read_csv("../data/app/app_nmf_16.csv")
# df = df.merge(app_nmf, on="id", how="left")

In [12]:
# app_info = pd.read_csv("../features/app/app_info.csv")
# df = df.merge(app_info, on="id", how="left")

app_graph

In [13]:
# to_pca = pd.read_csv("../features/app_graph/to_app_pca_mean.csv")
# from_pca = pd.read_csv("../features/app_graph/from_app_pca_mean.csv")

# df = df.merge(to_pca, on="id", how="left")
# df = df.merge(from_pca, on="id", how="left")

to_pca = pd.read_csv("../features/app_graph/to_app_pca_num_mean.csv")
from_pca = pd.read_csv("../features/app_graph/from_app_pca_num_mean.csv")

df = df.merge(to_pca, on="id", how="left")
df = df.merge(from_pca, on="id", how="left")

to_pca = pd.read_csv("../features/app_graph/to_app_pca_weight_mean.csv")
from_pca = pd.read_csv("../features/app_graph/from_app_pca_weight_mean.csv")

df = df.merge(to_pca, on="id", how="left")
df = df.merge(from_pca, on="id", how="left")

In [14]:
# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_num_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_num_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_weight_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_weight_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

In [15]:
# to_lda = pd.read_csv("../features/app_graph/to_app_lda_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

# to_lda = pd.read_csv("../features/app_graph/to_app_lda_num_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_num_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

# to_lda = pd.read_csv("../features/app_graph/to_app_lda_weight_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_weight_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

risk_graph

In [16]:
# to_risk = pd.read_csv("../features/risk_graph/to_risk_mean.csv")
# from_risk = pd.read_csv("../features/risk_graph/from_risk_mean.csv")

# df = df.merge(to_risk, on="id", how="left")
# df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_num_mean.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_num_mean.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_weight_mean.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_weight_mean.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

# to_risk = pd.read_csv("../features/risk_graph/to_risk_sum.csv")
# from_risk = pd.read_csv("../features/risk_graph/from_risk_sum.csv")

# df = df.merge(to_risk, on="id", how="left")
# df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_num_sum.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_num_sum.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_weight_sum.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_weight_sum.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

symbol_graph

In [17]:
# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_sum.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_sum.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_num_sum.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_num_sum.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

# to_cat_weight = pd.read_csv("../features/symbol_graph/to_symbol_weight_sum.csv")
# from_cat_weight = pd.read_csv("../features/symbol_graph/from_symbol_weight_sum.csv")

# df = df.merge(to_cat_weight, on="id", how="left")
# df = df.merge(from_cat_weight, on="id", how="left")

# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_mean.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_mean.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_num_mean.csv")
from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_num_mean.csv")

df = df.merge(to_cat_num, on="id", how="left")
df = df.merge(from_cat_num, on="id", how="left")

to_cat_weight = pd.read_csv("../features/symbol_graph/to_symbol_weight_mean.csv")
from_cat_weight = pd.read_csv("../features/symbol_graph/from_symbol_weight_mean.csv")

df = df.merge(to_cat_weight, on="id", how="left")
df = df.merge(from_cat_weight, on="id", how="left")

knn

In [18]:
# dp_knn = pd.read_csv("../features/knn/dp_knn_3.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_10.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_30.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_100.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_300.csv")
# df = df.merge(dp_knn, on="id", how="left")

中心性

In [19]:
pr = pd.read_csv("../features/graph/pagerank.csv")
df = df.merge(pr, on="id", how="left")

In [20]:
for d in ["to", "from"]:
    for w in ['', '_num', '_weight']:
        for f in ['sum', 'mean']:
            a = pd.read_csv("../features/graph/%s_pagerank%s_%s.csv" % (d, w, f))
            df = df.merge(a, on="id", how="left")

In [21]:
hits = pd.read_csv("../features/graph/hits.csv")
df = df.merge(hits, on="id", how="left")

for d in ["to", "from"]:
    for w in ['', '_num', '_weight']:
        for f in ['sum', 'mean']:
            a = pd.read_csv("../features/graph/%s_hits%s_%s.csv" % (d, w, f))
            df = df.merge(a, on="id", how="left")

二度联系人

In [22]:
# 联系人个数
for d1 in ["from", "to"]:
    for d2 in ["from", "to"]:
        for t in ["", "_num", "_weight"]:
            for f in ["sum", "mean"]:
                a = pd.read_csv("../features/graph/%s_d2_%s%s_%s.csv" % (d1, d2, t, f))
                df = df.merge(a, on="id", how="left")

In [23]:
# 二度联系人PageRank值
for d1 in ["from", "to"]:
    for d2 in ["from", "to"]:
        for t in ["num", "weight"]:
            for f in ["mean"]:
                a = pd.read_csv("../features/graph/%s_pg_%s_%s_%s_%s_%s.csv" % (d1, d2, t, f, t, f))
                df = df.merge(a, on="id", how="left")

In [24]:
# for d1 in ["from", "to"]:
#     for d2 in ["from", "to"]:
#         for t in ["num", "weight"]:
#             for f in ["sum", "mean"]:
#                 a = pd.read_csv("../features/graph/%s_hits_%s_%s_%s_%s_%s.csv" % (d1, d2, t, f, t, f))
#                 df = df.merge(a, on="id", how="left")

In [25]:
def  process_feature(train_x, valid_x, test_df):
    result = []
    drop_cols = ['id','label']
    for df in [train_x, valid_x, test_df]:
        result.append(df.drop(drop_cols, axis=1))
    return result 

In [32]:
import os
def cv(df, num_folds, param, model_dir, stratified=True, debug=False):
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    train_df = df[df.label.notnull()]
    test_df = df[df.label.isnull()]
    
    seed = 178
    if "seed" in param: 
        seed = param["seed"]
        del param['seed']
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
    oof_preds = np.zeros(train_df.shape[0])
    all_test_preds = []    
    feature_importance_df = pd.DataFrame()
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, train_df['label'])):
        train_x, train_y = train_df.iloc[train_idx], train_df['label'].iloc[train_idx]
        valid_x, valid_y = train_df.iloc[valid_idx], train_df['label'].iloc[valid_idx]
        fold_preds = test_df[["id"]]
        
        train_x, valid_x, test = process_feature(train_x, valid_x, test_df)
        if n_fold == 0:
            print(train_x.shape, valid_x.shape, test.shape)
        
        train_data = lgb.Dataset(train_x, label=train_y)
        validation_data = lgb.Dataset(valid_x, label=valid_y)

        clf=lgb.train(params,
                      train_data,
                      num_boost_round=10000,
                      valid_sets=[train_data, validation_data],
                      valid_names=["train", "valid"],
                      early_stopping_rounds=100,
                      verbose_eval=100)

        valid_preds = clf.predict(valid_x, num_iteration=clf.best_iteration)
        test_preds = clf.predict(test, num_iteration=clf.best_iteration)

        fold_preds['prob'] = test_preds
        fold_preds['fold_id'] = n_fold + 1
        all_test_preds.append(fold_preds)
        
        oof_preds[valid_idx] = valid_preds
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, valid_preds)))
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    full_auc = roc_auc_score(train_df['label'], oof_preds)
    print('Full AUC score %.6f' % full_auc)
    
    if not debug:
        train_df["prob"] = oof_preds
        train_df[['id', 'prob']].to_csv(model_dir + "pred_train.csv", index= False)

        all_test_preds = pd.concat(all_test_preds, axis=0)
        all_test_preds.to_csv(model_dir + "all_test_preds.csv", index=False)
        
        sub = pd.DataFrame()
        sub['id'] = all_test_preds.id.unique()
        sub.set_index("id", inplace=True)
        sub["prob"] = all_test_preds.groupby("id").prob.mean()
        sub.reset_index().to_csv(model_dir + "sub.txt", index=False)
    return full_auc

In [33]:
params_list =[
 {'boosting_type': 'goss', 'colsample_bytree': 0.6555004575000242, 'learning_rate': 0.016380004820033073, 'max_bin': 1000, 'metric': 'auc', 'min_child_weight': 2.1115838168176433, 'num_leaves': 108, 'reg_alpha': 23.247001339889128, 'reg_lambda': 997.9576062039534, 'subsample': 1.0, 'verbose': 1, 'seed': 4419},
 {'boosting_type': 'goss', 'colsample_bytree': 0.7187703092392053, 'learning_rate': 0.01939219215282862, 'max_bin': 310, 'metric': 'auc', 'min_child_weight': 2.66983907940641, 'num_leaves': 89, 'reg_alpha': 18.48224434106526, 'reg_lambda': 470.54675380054465, 'subsample': 1.0, 'verbose': 1, 'seed': 57},
 {'boosting_type': 'goss', 'colsample_bytree': 0.6274617979582582, 'learning_rate': 0.01680918441243103, 'max_bin': 780, 'metric': 'auc', 'min_child_weight': 0.8226071606806127, 'num_leaves': 73, 'reg_alpha': 14.466924422050258, 'reg_lambda': 658.5772060624658, 'subsample': 1.0, 'verbose': 1, 'seed': 1732},
 {'boosting_type': 'goss', 'colsample_bytree': 0.6991838451153098, 'learning_rate': 0.01577419276366034, 'max_bin': 940, 'metric': 'auc', 'min_child_weight': 7.758954855388241, 'num_leaves': 187, 'reg_alpha': 43.60868666926589, 'reg_lambda': 667.6371302027073, 'subsample': 1.0, 'verbose': 1, 'seed': 139},
 {'boosting_type': 'goss', 'colsample_bytree': 0.6198406436401038, 'learning_rate': 0.016662891953242748, 'max_bin': 820, 'metric': 'auc', 'min_child_weight': 3.5602833459924015, 'num_leaves': 77, 'reg_alpha': 13.398041512170746, 'reg_lambda': 631.8105595021391, 'subsample': 1.0, 'verbose': 1, 'seed': 7609}
]
result = []
for i, params in enumerate(params_list):
    model_dir = "../model_output/random/lgb_692/%d/" % i
    result.append(cv(df, 5, params, model_dir))

(15166, 692) (3793, 692) (6000, 692)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.859218	valid's auc: 0.664818
[200]	train's auc: 0.899888	valid's auc: 0.680354
[300]	train's auc: 0.925576	valid's auc: 0.688337
[400]	train's auc: 0.935431	valid's auc: 0.691704
[500]	train's auc: 0.952886	valid's auc: 0.694154
Early stopping, best iteration is:
[459]	train's auc: 0.947383	valid's auc: 0.695301


/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  1 AUC : 0.695301
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.859077	valid's auc: 0.651568
[200]	train's auc: 0.913734	valid's auc: 0.67877
[300]	train's auc: 0.944343	valid's auc: 0.685466
[400]	train's auc: 0.963459	valid's auc: 0.689933
[500]	train's auc: 0.976176	valid's auc: 0.692514
[600]	train's auc: 0.982721	valid's auc: 0.692794
Early stopping, best iteration is:
[582]	train's auc: 0.981517	valid's auc: 0.693517
Fold  2 AUC : 0.693517
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.867225	valid's auc: 0.669274
[200]	train's auc: 0.918294	valid's auc: 0.682883
[300]	train's auc: 0.948736	valid's auc: 0.690464
[400]	train's auc: 0.967062	valid's auc: 0.6936
[500]	train's auc: 0.978123	valid's auc: 0.69734
[600]	train's auc: 0.985444	valid's auc: 0.699051
[700]	train's auc: 0.990236	valid's auc: 0.701693
[800]	train's auc: 0.992685	valid's auc: 0.703438
[900]	train's auc: 0.99479	valid's auc: 0.70350

/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(15166, 692) (3793, 692) (6000, 692)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.907241	valid's auc: 0.678505
[200]	train's auc: 0.961796	valid's auc: 0.690657
[300]	train's auc: 0.982535	valid's auc: 0.69454
[400]	train's auc: 0.991616	valid's auc: 0.699601
[500]	train's auc: 0.9949	valid's auc: 0.697454
Early stopping, best iteration is:
[402]	train's auc: 0.991712	valid's auc: 0.699989
Fold  1 AUC : 0.699989
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.905605	valid's auc: 0.669526
[200]	train's auc: 0.962124	valid's auc: 0.684919
[300]	train's auc: 0.984293	valid's auc: 0.689683
[400]	train's auc: 0.992564	valid's auc: 0.69146
[500]	train's auc: 0.995538	valid's auc: 0.68968
Early stopping, best iteration is:
[423]	train's auc: 0.99313	valid's auc: 0.692928
Fold  2 AUC : 0.692928
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.909654	valid's auc: 0.674255
[200]	train's 

[500]	train's auc: 0.96756	valid's auc: 0.68799
[600]	train's auc: 0.980016	valid's auc: 0.688377
Early stopping, best iteration is:
[528]	train's auc: 0.972552	valid's auc: 0.689869
Fold  3 AUC : 0.689869
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.883269	valid's auc: 0.6547
[200]	train's auc: 0.944912	valid's auc: 0.67077
[300]	train's auc: 0.972397	valid's auc: 0.676043
[400]	train's auc: 0.984977	valid's auc: 0.679486
[500]	train's auc: 0.991268	valid's auc: 0.680593
[600]	train's auc: 0.994763	valid's auc: 0.68075
Early stopping, best iteration is:
[562]	train's auc: 0.993587	valid's auc: 0.68222
Fold  4 AUC : 0.682220
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.880314	valid's auc: 0.659506
[200]	train's auc: 0.943627	valid's auc: 0.676882
[300]	train's auc: 0.971658	valid's auc: 0.687622
[400]	train's auc: 0.984842	valid's auc: 0.694986
[500]	train's auc: 0.991408	valid's auc: 0.69841
[600]	train's au

In [34]:
result

[0.69561227872957154,
 0.69538706572468723,
 0.69491524070252297,
 0.69443423234221058,
 0.69421277468594611]

In [30]:
# params_list =[
#     {'boosting_type': 'goss', 'colsample_bytree': 0.6361074091768211, 'learning_rate': 0.017552101517925738, 'max_bin': 890, 'metric': 'auc', 'min_child_weight': 13.822743561422419, 'num_leaves': 142, 'reg_alpha': 44.02315295186438, 'reg_lambda': 786.7401929349503, 'subsample': 1.0, 'verbose': 1},
#     {'boosting_type': 'goss', 'colsample_bytree': 0.6242911020379066, 'learning_rate': 0.01653945448682752, 'max_bin': 680, 'metric': 'auc', 'min_child_weight': 12.650780009611925, 'num_leaves': 67, 'reg_alpha': 47.09175114406449, 'reg_lambda': 877.4856871282157, 'subsample': 1.0, 'verbose': 1},
#     {'boosting_type': 'goss', 'colsample_bytree': 0.6982064137489119, 'learning_rate': 0.016013714776900945, 'max_bin': 760, 'metric': 'auc', 'min_child_weight': 11.010544811253551, 'num_leaves': 133, 'reg_alpha': 37.6720426237371, 'reg_lambda': 952.6670482851789, 'subsample': 1.0, 'verbose': 1},
#     {'boosting_type': 'goss', 'colsample_bytree': 0.6053715371118441, 'learning_rate': 0.017157540317584066, 'max_bin': 840, 'metric': 'auc', 'min_child_weight': 8.338273524627725, 'num_leaves': 93, 'reg_alpha': 40.785833195844056, 'reg_lambda': 937.8635669398237, 'subsample': 1.0, 'verbose': 1},
#     {'boosting_type': 'goss', 'colsample_bytree': 0.666290363015605, 'learning_rate': 0.016595579965019504, 'max_bin': 830, 'metric': 'auc', 'min_child_weight': 10.620363000104456, 'num_leaves': 84, 'reg_alpha': 48.94431268173896, 'reg_lambda': 907.0034326430371, 'subsample': 1.0, 'verbose': 1}
# ]
# result = []
# for i, params in enumerate(params_list):
#     model_dir = "../model_output/lgb_692/%d/" % i
#     result.append(cv(df, 5, params, model_dir))

(15166, 692) (3793, 692) (6000, 692)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.853661	valid's auc: 0.654176
[200]	train's auc: 0.906743	valid's auc: 0.664989
[300]	train's auc: 0.935357	valid's auc: 0.667012
[400]	train's auc: 0.951788	valid's auc: 0.670769
[500]	train's auc: 0.964382	valid's auc: 0.672308
[600]	train's auc: 0.973723	valid's auc: 0.672276
[700]	train's auc: 0.979226	valid's auc: 0.674762
[800]	train's auc: 0.983855	valid's auc: 0.673806
Early stopping, best iteration is:
[722]	train's auc: 0.980246	valid's auc: 0.675531


/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  1 AUC : 0.675531
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.847351	valid's auc: 0.657162
[200]	train's auc: 0.901504	valid's auc: 0.672748
[300]	train's auc: 0.930933	valid's auc: 0.678723
[400]	train's auc: 0.950797	valid's auc: 0.679203
[500]	train's auc: 0.964722	valid's auc: 0.682059
[600]	train's auc: 0.973597	valid's auc: 0.683405
[700]	train's auc: 0.979807	valid's auc: 0.684176
[800]	train's auc: 0.983714	valid's auc: 0.687134
[900]	train's auc: 0.987407	valid's auc: 0.685959
[1000]	train's auc: 0.990062	valid's auc: 0.686448
Early stopping, best iteration is:
[968]	train's auc: 0.989184	valid's auc: 0.687782
Fold  2 AUC : 0.687782
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.834631	valid's auc: 0.683912
[200]	train's auc: 0.877965	valid's auc: 0.696431
[300]	train's auc: 0.89945	valid's auc: 0.702992
[400]	train's auc: 0.908603	valid's auc: 0.704318
[500]	train's auc: 0.927063	valid's auc: 0.

/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(15166, 692) (3793, 692) (6000, 692)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.8306	valid's auc: 0.645728
[200]	train's auc: 0.88896	valid's auc: 0.661354
[300]	train's auc: 0.919992	valid's auc: 0.666546
[400]	train's auc: 0.939654	valid's auc: 0.667952
[500]	train's auc: 0.952925	valid's auc: 0.669842
[600]	train's auc: 0.962737	valid's auc: 0.670497
[700]	train's auc: 0.96925	valid's auc: 0.67108
[800]	train's auc: 0.975446	valid's auc: 0.671137
[900]	train's auc: 0.980117	valid's auc: 0.673126
[1000]	train's auc: 0.982882	valid's auc: 0.672272
Early stopping, best iteration is:
[929]	train's auc: 0.981132	valid's auc: 0.674513
Fold  1 AUC : 0.674513
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.827414	valid's auc: 0.655286
[200]	train's auc: 0.883765	valid's auc: 0.676109
[300]	train's auc: 0.915889	valid's auc: 0.682726
[400]	train's auc: 0.937279	valid's auc: 0.684657
[500]	train's auc: 0.952128	valid

[600]	train's auc: 0.969313	valid's auc: 0.69347
[700]	train's auc: 0.97676	valid's auc: 0.696103
[800]	train's auc: 0.981923	valid's auc: 0.696763
[900]	train's auc: 0.986071	valid's auc: 0.700438
Early stopping, best iteration is:
[898]	train's auc: 0.985991	valid's auc: 0.700757
Fold  5 AUC : 0.700757
Full AUC score 0.691944


FileNotFoundError: [Errno 2] No such file or directory: '../model_output/lgb_692/3/pred_train.csv'